Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [29]:

import copy
from dataclasses import dataclass
from random import choice, randint
import random


import lab9_lib

 # population initialized here
μ = 50  #population size
λ = 200 #offspring size
σ = 0.001   
MUTATION_PROBABILITY = 0.15
TOURNAMENT_SIZE = 2
GEN_SIZE = 50

In [15]:
## you should try with all the possibile optimisations in order to creare a population with a fitness as much closer to 100% as possible


#the genome is fixed (0 and 1 sequence)

#define the mutation

#define the parent selection -> apply the optimisations

#define the survival selection -> apply the optimisations



In [30]:
@dataclass
class Individual:
    fitness: tuple
    genotype: list[bool]

def select_parent(pop):
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion

def mutate(ind: Individual) -> Individual:
    offspring = copy(ind)
    pos = randint(0, GEN_SIZE-1)
    offspring.genotype[pos] = not offspring.genotype[pos]
    offspring.fitness = None
    return offspring

def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = randint(0, GEN_SIZE-1)
    offspring = Individual(fitness=None,
                           genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:])
    assert len(offspring.genotype) == GEN_SIZE
    return offspring

In [31]:
# GENERAL FUNCTIONS, USEFUL FOR ALL THE APPROACHES
PROB_INSTANCE = 2

def binary_to_string(genome):
    return ''.join(str(g) for g in genome)

def fitness(genome):
    return lab9_lib.make_problem(PROB_INSTANCE)(genome)

def local_search(population,pop_size,off_size):
    offspring = list()
    for _ in range(off_size):
        if random() < MUTATION_PROBABILITY:  # self-adapt mutation probability
            # mutation  # add more clever mutations
            p = select_parent(population)
            o = mutate(p)
        else:
            # xover # add more xovers
            p1 = select_parent(population)
            p2 = select_parent(population)
            o = one_cut_xover(p1, p2)
        offspring.append(o)

    for i in offspring:
        i.fitness = fitness(i.genotype)
    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:pop_size]
    print(population[0].fitness)



In [32]:
from copy import deepcopy
import random
import lab9_lib

class Island:

    def __init__(self, population_size,off_size,max_iter):
        self.population = []
        self.best_individual = None
        self.best_fitness = None
        self.population_size = population_size
        self.max_iterations = max_iter
        self.off_size = off_size

    def gen_population(self, population_cpy : list):
        for _ in range(self.population_size):
            individual = []
            for _ in range(len(individual)):
                individual.append(random.randint(0, 1))
            self.population.append(individual)
            population_cpy.remove(individual)
        return population_cpy       #return the updated variable -> the picked individuals have been removed

    def evolve(self):
        for _ in range(self.max_iterations):
            local_search(self.population,self.population_size,self.off_size)
        self.best_fitness =  self.get_best_fitness()
        self.best_individual =  self.get_best_individual()

    def get_best_fitness(self):
        if self.best_fitness is None:
            self.best_fitness = float("-inf")
        for individual in self.population:
            fitness = fitness(individual.genotype)
            if fitness > self.best_fitness:
                self.best_fitness = fitness
        return self.best_fitness

    def get_best_individual(self):
        if self.best_individual is None:
            self.best_fitness = float("-inf")
        for individual in self.population:
            fitness = fitness(individual.genotype)
            if fitness > self.best_fitness:
                self.best_fitness = fitness
                self.best_individual = individual
        return self.best_individual


    def __repr__(self):
        return f"Island(population={self.population}, best_individual={self.best_individual}, best_fitness={self.best_fitness})"
    
             ####################### end of class Island implementation #######################



def island_model(population,population_size, island_size, max_iterations,island_max_iter):
    best_fitness = 0.0
    best_genome = None

    island_count = population_size // island_size

    islands = []
    pop_cpy = deepcopy(population)

    #generation and population of islands
    for _ in range(island_count):
        island = Island(island_size,island_size*10 ,island_max_iter)
        pop_cpy = island.gen_population(pop_cpy)
        islands.append(island)
        


    for _ in range(max_iterations):
        #each island evolves on its own, indipendently
        for island in islands:
            island.evolve()

        # exchange the best individuals according to fitness -> migration concept applied here -> used to improve fitness and bring new info from another search space area
        for i in range(island_count // 2):
            islands[i].best_individual, islands[island_count - 1 - i].best_individual = islands[island_count - 1 - i].get_best_individual(), islands[i].get_best_individual()

        #find the best fitness among the several islands
        for island in islands:
            print(island)
            if island.best_fitness > best_fitness:
                best_fitness = island.best_fitness
                best_genome = binary_to_string(island.best_individual)

    return best_fitness, best_genome






In [33]:
 # population initialized here
μ = 50  #population size
λ = 200 #offspring size
σ = 0.001   

# import the necessary modules
import random
import lab9_lib

# define the input parameters
population_size = 100
island_size = 20
max_iterations = 100
island_max_iter = 50

# generate the initial population
population = [
    Individual(
        genotype=[choice((False, False)) for _ in range(GEN_SIZE)],
        fitness=None,
    )
    for _ in range(population_size)
]

# run the island model
best_fitness, best_genome = island_model(population, population_size, island_size, max_iterations, island_max_iter)

# print the results
print("Best fitness:", best_fitness)
print("Best genome:", best_genome)



ValueError: list.remove(x): x not in list

In [2]:
fitness = lab9_lib.make_problem(10)
for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

10100011100011010000111011110010111110101011101100: 9.11%
00100110000001100001100101101100111010100010111101: 7.33%
01011111110101011011110010011000101101001101111000: 23.33%
11011110011000110101111101101101101011010100010101: 9.11%
01101000111110011100011011110100111011011111111001: 9.11%
11000100110001001001111100011010011000001111010111: 15.33%
01000110100000011000000101011001100100000011000000: 7.56%
10111000100100000000110001100011110000010001011111: 17.56%
00011000001001111101100100001111000000100101111100: 7.34%
01111011100110011111110100101000000100011101010111: 15.33%
10
